**This notebook were used on Google Colab.**

In [ ]:
!pip install tensorflow==2.19 ai_edge_litert==2.0.3 -q

import tensorflow as tf
print("Tensorflow version:", tf.__version__)
import tf_keras
print("Tensorflow Keras version:", tf_keras.__version__)
import ai_edge_litert
print("ai edge litert version:", ai_edge_litert.__version__)
import tensorflow_hub as hub
print("Tensorflow Hub version:", hub.__version__)
import tensorflow_datasets as tfds
print("Tensorflow Datasets version:", tfds.__version__)
import numpy as np
print("NumPy version:", np.__version__)
import os

!wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz     # download data
!mkdir -p data                                                                                     # create data folder
!tar -xzvf speech_commands_v0.02.tar.gz -C data                                                    # unzip data to data/
!rm speech_commands_v0.02.tar.gz                                                                   # remove tar.gz file

A saída de streaming foi truncada nas últimas 5000 linhas.
./on/fc3ba625_nohash_0.wav
./on/db9d52d2_nohash_0.wav
./on/72242187_nohash_4.wav
./on/b0f5b16d_nohash_0.wav
./on/d9aa8c90_nohash_1.wav
./on/525eaa62_nohash_0.wav
./on/42e3f068_nohash_0.wav
./on/94d370bf_nohash_4.wav
./on/bf8d5617_nohash_0.wav
./on/e2b7d6d6_nohash_2.wav
./on/89ed36ab_nohash_0.wav
./on/30276d03_nohash_1.wav
./on/364c0853_nohash_0.wav
./on/7cbf645a_nohash_0.wav
./on/c98bbbd3_nohash_2.wav
./on/9a7c1f83_nohash_5.wav
./on/3589bc72_nohash_0.wav
./on/3ae5c04f_nohash_0.wav
./on/adc216c5_nohash_2.wav
./on/c50f55b8_nohash_15.wav
./on/324210dd_nohash_2.wav
./on/c120e80e_nohash_2.wav
./on/69086eb0_nohash_0.wav
./on/dce05676_nohash_1.wav
./on/435695e3_nohash_1.wav
./on/179a61b7_nohash_1.wav
./on/f5341341_nohash_0.wav
./on/91b03183_nohash_0.wav
./on/96d8bb6f_nohash_2.wav
./on/6a1908f8_nohash_0.wav
./on/6ace4fe1_nohash_1.wav
./on/84d1e469_nohash_1.wav
./on/ba676390_nohash_0.wav
./on/1dc86f91_nohash_4.wav
./on/3a929277_nohash_2

In [ ]:
import pandas as pd

categories: list[str] = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "bed", "bird", "cat", "dog", "happy", "house", "marvin", "sheila", "tree", "wow"]
mapping: dict[str, str] = {category:files for category, files in zip(categories, [os.listdir(f"data/{category}/") for category in categories])}
"""
mapping is a dict of this type:
{
    'yes': ['0a7c2a8d_nohash_0.wav', '0a7c2a8d_nohash_1.wav', ... , '0a7c2a8d_nohash_99.wav'],
    'no': ['0a7c2a8d_nohash_0.wav', '0a7c2a8d_nohash_1.wav',  ... , '0a7c2a8d_nohash_99.wav'],
    'up': ['0a7c2a8d_nohash_0.wav', '0a7c2a8d_nohash_1.wav',  ... , '0a7c2a8d_nohash_99.wav'],
    ...
    'wow': ['0a7c2a8d_nohash_0.wav', '0a7c2a8d_nohash_1.wav',  ... , '0a7c2a8d_nohash_99.wav'],
}
"""

rows: list[dict[str, str]] = [
    {
        "category": category,
        "path": f"data/{category}/{username}"
    }
    for category, usernames in mapping.items()
    for username in usernames
]

df = pd.DataFrame(rows)
df.head()

target_cat: list[str] = ["up", "down", "left", "right", "on", "off", "stop", "go"]
df = df[df["category"].isin(target_cat)].copy().reset_index(drop=True)

class_to_id = {c: i for i, c in enumerate(target_cat)}

paths = df["path"].astype(str).values
labels = df["category"].map(class_to_id).astype(np.int32).values

print(paths[:5])
print(labels[:5])

df.head()

['data/up/72aa7259_nohash_0.wav' 'data/up/6ac35824_nohash_0.wav'
 'data/up/ccca5655_nohash_2.wav' 'data/up/850e2222_nohash_2.wav'
 'data/up/a7acbbeb_nohash_1.wav']
[0 0 0 0 0]


,category,path
0,up,data/up/72aa7259_nohash_0.wav
1,up,data/up/6ac35824_nohash_0.wav
2,up,data/up/ccca5655_nohash_2.wav
3,up,data/up/850e2222_nohash_2.wav
4,up,data/up/a7acbbeb_nohash_1.wav


In [ ]:
# get train, validation, and test datasets
from sklearn.model_selection import train_test_split

paths = np.array(paths)
labels = np.array(labels)

# 1) Train vs (val+test): 80% / 20%, stratified
paths_train, paths_temp, labels_train, labels_temp = train_test_split(
    paths,
    labels,
    test_size=0.2,          # 20% -> will become 10% val + 10% test
    stratify=labels,
    random_state=42,
)

# 2) (val+test) -> val and test: 50% / 50%, stratified
paths_val, paths_test, labels_val, labels_test = train_test_split(
    paths_temp,
    labels_temp,
    test_size=0.5,          # 0.5 * 20% = 10% of total
    stratify=labels_temp,
    random_state=42,
)

In [ ]:
import tensorflow as tf
from tf_keras import Sequential
from tf_keras.layers import Input, Conv2D, ReLU, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, DepthwiseConv2D, Add
from tf_keras.initializers import TruncatedNormal
from tf_keras.models import Model
from tf_keras.optimizers import Adam
from tf_keras.losses import SparseCategoricalCrossentropy
from tf_keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from typing import Literal
from ai_edge_litert.interpreter import Interpreter
from pathlib import Path
import pandas as pd
import json
import os
import time


In [ ]:
def preprocess(paths, labels, config, is_training=True):

    def load_wav_16k_mono(path):
        file_contents = tf.io.read_file(path)
        wav, sr = tf.audio.decode_wav(file_contents, desired_channels=1)
        wav = tf.squeeze(wav, axis=-1)
        # Pad/Crop to exactly 1 second (16000 samples)
        wav = wav[:16000]
        zero_padding = tf.zeros([16000] - tf.shape(wav), dtype=tf.float32)
        wav = tf.concat([wav, zero_padding], 0)
        return wav

    def preprocess_with_config(path, label):
        waveform = load_wav_16k_mono(path)

        # Dynamic MFCC generation based on 'config'
        stft = tf.signal.stft(
            waveform,
            frame_length=config['frame_length'],
            frame_step=config['frame_step'],
            fft_length=config['fft_length']
        )
        spectrogram = tf.abs(stft)

        num_spectrogram_bins = spectrogram.shape[-1]

        mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=config['num_mel_bins'],
            num_spectrogram_bins=num_spectrogram_bins,
            sample_rate=16000,
            lower_edge_hertz=config['low_freq'],
            upper_edge_hertz=config['high_freq']
        )

        mel_spectrogram = tf.tensordot(spectrogram, mel_weight_matrix, 1)
        mel_spectrogram.set_shape(spectrogram.shape[:-1].concatenate(
            tf.TensorShape([config['num_mel_bins']])))

        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)

        # Slice to keep only the desired number of MFCCs
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(
            log_mel_spectrogram)[..., :config['num_mfccs']]

        mfccs = tf.expand_dims(mfccs, -1) # Add channel dim

        return mfccs, label

    # Build Dataset
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if is_training:
        ds = ds.shuffle(len(paths))

    ds = ds.map(preprocess_with_config, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
def build_edge_kws_model(input_shape, label_count=8, alpha=1.0):
    """
    input_shape: Tuple, e.g., (49, 10, 1) -> (Time, MFCCs, Channels)
    num_classes: 8 (stop, go, left, right, up, down, on, off)
    alpha: Width multiplier. 1.0 is baseline. 0.5 cuts filters in half (smaller model).
    """

    def depthwise_separable_block(x, filters, stride=1):
        """
        Creates a block: Depthwise Conv -> BatchNorm -> ReLU -> Pointwise Conv -> BatchNorm -> ReLU
        This is the core building block of MobileNet.
        """

        # 1. Depthwise Convolution (Spatial features)
        # stride to reduce dimension (downsampling) instead of MaxPool
        x = DepthwiseConv2D(kernel_size=(3, 3),
                                  padding='same',
                                  strides=stride,
                                  use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        # 2. Pointwise Convolution (Channel mixing)
        # 1x1 convolution to project to new number of filters
        x = Conv2D(filters, kernel_size=(1, 1),
                          padding='same',
                          use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        return x

    inputs = Input(shape=input_shape, name="fingerprint_input")

    # --- Initial Standard Conv ---
    # start with one standard Conv2D to get initial features
    x = Conv2D(int(16 * alpha), (3, 3),
                      padding='same',
                      strides=2, # Initial downsampling
                      use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # --- Depthwise Separable Blocks ---
    # Block 1
    x = depthwise_separable_block(x, int(32 * alpha), stride=1)

    # Block 2 (Downsample here)
    x = depthwise_separable_block(x, int(48 * alpha), stride=2)

    # Block 3
    x = depthwise_separable_block(x, int(64 * alpha), stride=1)

    # Block 4 (Downsample here)
    x = depthwise_separable_block(x, int(96 * alpha), stride=2)

    # --- Classification Head ---
    x = GlobalAveragePooling2D()(x)

    # Dropout helps prevent overfitting on small datasets
    x = Dropout(0.2)(x)

    outputs = Dense(label_count, activation=None)(x)

    model = Model(inputs, outputs)
    return model

def build_mobilenet_v2_kws(input_shape, label_count=8, alpha=1.0):

    def inverted_residual_block(x, filters, stride, expansion=4):
        """
        Block like MobileNetV2:
        1. Expansion (1x1 Conv) -> Increases the number of channels/dimensions to 'think'
        2. Depthwise (3x3 Conv) -> Processes the features spatially with minimal computation
        3. Projection (1x1 Conv) -> Compresses the dimensions back down (Linear Bottleneck)
        4. Residual (Add) -> Adds the input to the output if dimensions match (stride=1)
        """
        inputs = x
        in_channels = inputs.shape[-1]

        # 1. Exansion
        # Inceases the number of channels temporarily
        x = Conv2D(expansion * in_channels, kernel_size=1, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        # 2. Depthwise Convolution
        x = DepthwiseConv2D(kernel_size=3, strides=stride, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        # 3. Projection (Pointwise Linear)
        # Here we do not use ReLU at the end (Linear Bottleneck)
        x = Conv2D(filters, kernel_size=1, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)

        # 4. Residual Connection (Skip Connection)
        # We can only add if the size (stride=1) and the channels (in=out) are equal.
        if stride == 1 and in_channels == filters:
            x = Add()([inputs, x])

        return x

    inputs = Input(shape=input_shape)

    # Initial Layer
    # Initial filters slightly increased to better capture the raw audio.
    first_filters = int(32 * alpha)
    x = Conv2D(first_filters, kernel_size=3, strides=2, padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # --- Residual Blocks (Inverted Bottlenecks) ---
    # You can add or remove blocks to adjust the size

    # Block 1: Stride 1 (Keeps size, refine features)
    x = inverted_residual_block(x, filters=int(16 * alpha), stride=1, expansion=1)

    # Block 2: Stride 2 (Decrease size, learn more complex patterns)
    x = inverted_residual_block(x, filters=int(24 * alpha), stride=2, expansion=6)
    x = inverted_residual_block(x, filters=int(24 * alpha), stride=1, expansion=6)

    # Block 3: Stride 2 (Decrease size)
    x = inverted_residual_block(x, filters=int(32 * alpha), stride=2, expansion=6)
    x = inverted_residual_block(x, filters=int(32 * alpha), stride=1, expansion=6)

    # Block 4: Stride 1 (Final refinement)
    x = inverted_residual_block(x, filters=int(64 * alpha), stride=1, expansion=6)
    x = inverted_residual_block(x, filters=int(64 * alpha), stride=1, expansion=6)

    # --- Classification ---
    x = Conv2D(int(128 * alpha), kernel_size=1, use_bias=False)(x) # Final Expansion
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    outputs = Dense(label_count, activation=None)(x)

    model = Model(inputs, outputs)
    return model

def get_model(model_name: str, input_shape: tuple):
    match model_name:
        case 'depthwise_separable_convs':
            return build_edge_kws_model(input_shape, alpha=0.5)
        case 'build_mobilenet_v2_kws':
            return build_mobilenet_v2_kws(input_shape, alpha=0.5)
        case _:
            raise ValueError(f'Unknown model: {model}')

In [ ]:
PLOTS_DIR: str = "plots"

def save_plot_history(history, id_normal_model: int, architecture: str):
    os.makedirs(PLOTS_DIR, exist_ok=True)

    hist = history.history

    plt.figure(figsize=(10, 4))
    plt.suptitle(f"{architecture}")

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(hist["loss"], label="train loss")
    if "val_loss" in hist:
        plt.plot(hist["val_loss"], label="val loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(hist["accuracy"], label="train acc")
    if "val_accuracy" in hist:
        plt.plot(hist["val_accuracy"], label="val acc")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout for the suptitle
    plt.savefig(os.path.join(PLOTS_DIR, f"{str(id_normal_model)}.pdf"))
    plt.savefig(os.path.join(PLOTS_DIR, f"{str(id_normal_model)}.svg"))
    plt.close()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def evaluate_tflite_model(id_lite_model: int, test_ds, tflite_path: str) -> tuple[float, list[str]]:
    """
    Returns the accuracy of the tflite model and the list of paths of wrong audios.
    """
    interpreter = Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_index = input_details[0]['index']
    output_index = output_details[0]['index']

    predictions = []
    test_labels, test_imgs = [], []
    for img, label in test_ds:
        for i in range(img.shape[0]):
            img_single = img[i].numpy()[None, ...]  # shape (1, ...)
            interpreter.set_tensor(input_index, img_single)
            interpreter.invoke()
            output = interpreter.get_tensor(output_index)
            predictions.append(output)
            test_labels.append(label[i].numpy())

    predictions_labels = []
    wrong_audio_paths = []
    score = 0
    count = 0   # to get the path of the audio
    for item in range(0,len(predictions)):
        prediction=np.argmax(predictions[item])
        predictions_labels.append(prediction)
        label = test_labels[item]
        if prediction==label:
            score=score+1
        else:
            wrong_audio_paths.append(paths_test[count])
        count += 1

    target_cat: list[str] = ["up", "down", "left", "right", "on", "off", "stop", "go"]
    cm = confusion_matrix(test_labels, predictions_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_cat)
    disp.plot(cmap=plt.cm.Blues)
    plt.xlabel("Predicted")
    plt.ylabel("Real")
    plt.title("Confusion Matrix")
    plt.savefig(f"plots/cm_{id_lite_model}.pdf")
    plt.savefig(f"plots/cm_{id_lite_model}.svg")
    plt.close()
    return score/len(predictions), wrong_audio_paths

In [ ]:
from google.colab import files

def calculate_time_frames(config: dict) -> int:
    """Calculates the time axis dimension for the Keras input shape."""
    TOTAL_SAMPLES = 16000

    frame_length = config['frame_length']
    frame_step = config['frame_step']

    # Calculate frames using the STFT formula (using floor division // in Python)
    time_frames = (TOTAL_SAMPLES - frame_length) // frame_step + 1
    return time_frames

def pipeline(id_normal_model: int, id_lite_model: int, config: dict, architecture: str, epochs: int):
    """
    Orchestrates the full training, conversion, and evaluation loop for Edge AI models.

    Args:
        config (dict): Dictionary containing preprocessing and training hyperparameters.
        architecture (str): Model function name.
    """

    # ensure reproducibility
    tf_keras.utils.set_random_seed(36)

    # ==========================================
    # 1. SETUP & DIRECTORIES
    # ==========================================
    print("\n--- 1. Initializing Pipeline ---")

    # Create directories if they don't exist
    models_dir = Path("models/")
    models_dir.mkdir(exist_ok=True)

    # Files to store metrics and errors
    results_csv_path = "results.csv"
    wrong_audios_json_path = "wrong_audios.json"

    # Initialize or load the Results DataFrame
    if os.path.exists(results_csv_path):
        df_results = pd.read_csv(results_csv_path)
    else:
        df_results = pd.DataFrame(columns=["id", "architecture", "model_type", "test_accuracy", "size_bytes", "training_time","high_freq", "num_mfccs", "frame_length", "frame_step", "num_mel_bins"])

    # ==========================================
    # 2. DATA PREPROCESSING
    # ==========================================
    print("--- 2. Preprocessing Data ---")
    # We prepare datasets once using the passed 'config'
    # Note: 'preprocess' must be defined elsewhere in your code
    train_ds = preprocess(paths_train, labels_train, config, is_training=True)
    val_ds   = preprocess(paths_val,   labels_val,   config, is_training=False)
    test_ds  = preprocess(paths_test,  labels_test,  config, is_training=False)

    # ==========================================
    # 3. MAIN TRAINING LOOP
    # ==========================================

    print(f"\n" + "="*40)
    print(f"PROCESSING ARCHITECTURE: {architecture}")
    print("="*40)

    # --------------------------------------
    # A. Build & Compile
    # --------------------------------------
    print(f"Step A: Building and Compiling {architecture}...")
    # Assuming 'get_model' is a helper function you have that returns a Keras model
    time_frames = calculate_time_frames(config)
    model = get_model(model_name=architecture, input_shape=(time_frames, config['num_mfccs'], 1))

    # EarlyStopping prevents overfitting and saves time
    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=10,               # Increased patience slightly for stability
        restore_best_weights=True  # CRITICAL: Keep the best version, not the last
    )

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        # from_logits=True is numerically more stable for Softmax output
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    # --------------------------------------
    # B. Train
    # --------------------------------------
    print(f"Step B: Training {architecture}...")
    t0 = time.time()
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[early_stop],
        verbose=1
    )
    t1 = time.time()
    # Save the training graph (helper function)
    save_plot_history(history, id_normal_model=id_normal_model,  architecture=architecture)

    # --------------------------------------
    # C. Evaluate Normal Model (Float32)
    # --------------------------------------
    print(f"Step C: Evaluating Normal Keras Model...")
    _, normal_acc = model.evaluate(test_ds, verbose=0)
    print(f"   -> Accuracy: {normal_acc:.2%}")

    # Save the standard Keras model (needed for conversion)
    normal_model_path = models_dir / str(id_normal_model)
    model.save(normal_model_path, save_format="tf")

    # --------------------------------------
    # D. Convert to TFLite (Quantization)
    # --------------------------------------
    print(f"Step D: Converting to TFLite (Optimization)...")

    tflite_path = models_dir / f"{str(id_lite_model)}.tflite"

    # Load the saved model into a Converter
    converter = tf.lite.TFLiteConverter.from_saved_model(str(normal_model_path))

    # Apply Optimizations (Quantization) - Crucial for size reduction (200kb -> 50kb)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    # Convert and Save
    tflite_model = converter.convert()
    tflite_size_bytes = tflite_path.write_bytes(tflite_model)

    print(f"   -> Model saved to: {tflite_path}")
    print(f"   -> Size: {tflite_size_bytes / 1024:.2f} KB")

    # --------------------------------------
    # E. Evaluate TFLite Model
    # --------------------------------------
    print(f"Step E: Evaluating TFLite Model...")

    # Note: We pass the path to the TFLite model so the evaluator can load the interpreter
    lite_acc, wrong_audios = evaluate_tflite_model(id_lite_model, test_ds, str(tflite_path))
    print(f"   -> TFLite Accuracy: {lite_acc:.2%}")

    # --------------------------------------
    # F. Log Results & Errors
    # --------------------------------------
    print(f"Step F: Logging Results...")
    # 1. Update CSV DataFrame
    new_rows = [
        {
            "id": id_normal_model,
            "architecture": architecture,
            "model_type": "normal",
            "test_accuracy": normal_acc,
            "size_bytes": 0, # Standard models are huge, we usually ignore this size for Edge
            "training_time": t1-t0,
            "high_freq": config['high_freq'],
            "num_mfccs": config['num_mfccs'],
            "frame_length": config['frame_length'],
            "frame_step": config['frame_step'],
            "num_mel_bins": config['num_mel_bins']
        },
        {
            "id": id_lite_model,
            "architecture": architecture,
            "model_type": "lite (quantized)",
            "test_accuracy": lite_acc,
            "size_bytes": tflite_size_bytes,
            "training_time": 0, # lite model is not trained, only converted from previous one
            "high_freq": config['high_freq'],
            "num_mfccs": config['num_mfccs'],
            "frame_length": config['frame_length'],
            "frame_step": config['frame_step'],
            "num_mel_bins": config['num_mel_bins']
        }
    ]

    new_results_df = pd.DataFrame(new_rows)

    # FIX: Check if the existing dataframe is empty to avoid the FutureWarning
    if df_results.empty:
        df_results = new_results_df
    else:
        df_results = pd.concat([df_results, new_results_df], ignore_index=True)

    df_results.to_csv(results_csv_path, index=False)

    # 2. Update JSON with wrong predictions
    # Initialize list if new architecture
    if os.path.exists(wrong_audios_json_path):
        with open(wrong_audios_json_path, 'r') as f:
            wrong_audios_dict = json.load(f)
        wrong_audios_dict[id_lite_model] = {'info': {}, 'audios_path': []}
    else:
        wrong_audios_dict = {id_lite_model: {'info': {}, 'audios_path': []}}

    wrong_audios_dict[id_lite_model]['audios_path'].extend(wrong_audios)
    wrong_audios_dict[id_lite_model]['info'] = new_rows[-1]

    with open(wrong_audios_json_path, 'w') as f:
        json.dump(wrong_audios_dict, f, indent=4)

    # --------------------------------------
    # G. Download Results
    # --------------------------------------
    print("Downloading results...")

    # this is inside this loop because colab has a free tier limit and now we are always downloading the most up to date results
    !zip -r models.zip models/
    !zip -r plots.zip plots/

    files.download('results.csv')
    files.download('wrong_audios.json')
    files.download('models.zip')
    files.download('plots.zip')

    print("\n" + "="*40)
    print("PIPELINE FINISHED SUCCESSFULLY")
    print(f"Results saved to: {results_csv_path}")
    print("="*40)

In [ ]:
!rm -r "results.csv" "models" "plots" "wrong_audios.json"

In [ ]:
!unzip -q models.zip
!unzip -q plots.zip

In [ ]:
%%time

tf.config.experimental.enable_op_determinism()

architectures: list[str] = ['depthwise_separable_convs', 'build_mobilenet_v2_kws']

param_grid = [
    # Baseline
    {'high_freq': 4000.0, 'num_mfccs': 13, 'frame_length': 640, 'frame_step': 320, 'num_mel_bins': 40, 'batch_size': 64, 'fft_length': 1024, 'low_freq': 20.0},

    {'high_freq': 8000.0, 'num_mfccs': 13, 'frame_length': 640, 'frame_step': 320, 'num_mel_bins': 40, 'batch_size': 64, 'fft_length': 1024, 'low_freq': 20.0},

    {'high_freq': 8000.0, 'num_mfccs': 13, 'frame_length': 480, 'frame_step': 320, 'num_mel_bins': 40, 'batch_size': 64, 'fft_length': 1024, 'low_freq': 20.0},

    {'high_freq': 8000.0, 'num_mfccs': 23, 'frame_length': 480, 'frame_step': 240, 'num_mel_bins': 40, 'batch_size': 64, 'fft_length': 1024, 'low_freq': 20.0},

    {'high_freq': 8000.0, 'num_mfccs': 23, 'frame_length': 320, 'frame_step': 240, 'num_mel_bins': 40, 'batch_size': 64, 'fft_length': 1024, 'low_freq': 20.0},

    {'high_freq': 8000.0, 'num_mfccs': 30, 'frame_length': 320, 'frame_step': 240, 'num_mel_bins': 40, 'batch_size': 64, 'fft_length': 1024, 'low_freq': 20.0},
]


grid_idx = 5                # idx of param_grid
count = 4 * grid_idx        # "count" variable is the start index
# each group values of param_grid will produce 4 models (1 normal + 1 lite for each of the 2 architectures)
for config in [param_grid[grid_idx]]:
    for architecture in architectures:
        pipeline(id_normal_model=count, id_lite_model=count+1, config=config, architecture=architecture, epochs=100)
        count += 2


--- 1. Initializing Pipeline ---
--- 2. Preprocessing Data ---

PROCESSING ARCHITECTURE: depthwise_separable_convs
Step A: Building and Compiling depthwise_separable_convs...
Step B: Training depthwise_separable_convs...
Epoch 1/100
382/382 [==============================] - 51s 126ms/step - loss: 1.7594 - accuracy: 0.3620 - val_loss: 1.7727 - val_accuracy: 0.4604
Epoch 2/100
382/382 [==============================] - 47s 124ms/step - loss: 1.0273 - accuracy: 0.6680 - val_loss: 0.8099 - val_accuracy: 0.7503
Epoch 3/100
382/382 [==============================] - 47s 123ms/step - loss: 0.7085 - accuracy: 0.7725 - val_loss: 0.5709 - val_accuracy: 0.8168
Epoch 4/100
382/382 [==============================] - 46s 120ms/step - loss: 0.5519 - accuracy: 0.8241 - val_loss: 0.5026 - val_accuracy: 0.8338
Epoch 5/100
382/382 [==============================] - 49s 129ms/step - loss: 0.4586 - accuracy: 0.8544 - val_loss: 0.4288 - val_accuracy: 0.8635
Epoch 6/100
382/382 [===========================

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


PIPELINE FINISHED SUCCESSFULLY
Results saved to: results.csv
CPU times: user 2h 9min 43s, sys: 5min 18s, total: 2h 15min 1s
Wall time: 1h 39min 20s
